In [7]:
import numpy as np

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D

# Define the input shape
input_shape = (395, 488, 3)

# Create the input layer
input_layer = Input(shape=input_shape)

# Load the InceptionResNetV2 model
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_tensor=input_layer)

# Freeze the pre-trained weights
for layer in base_model.layers:
    layer.trainable = False

# Add a global average pooling layer
x = GlobalAveragePooling2D()(base_model.output)

# Add a dense layer with 512 units and ReLU activation
x = Dense(512, activation='relu')(x)

# Add the output layer with sigmoid activation
output_layer = Dense(1, activation='sigmoid')(x)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 395, 488, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 197, 243, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 197, 243, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [2]:
# Path to the images
path_to_images = "training-data/dataset1/known_images"
train_dataset, val_dataset = keras.utils.image_dataset_from_directory(
    path_to_images,
    class_names=["normal", "sick"],
    batch_size=32,
    image_size=(395, 488),
    validation_split=0.2,
    subset="both",
    seed=200
)

Found 715 files belonging to 2 classes.
Using 572 files for training.
Using 143 files for validation.


In [8]:
def get_features_and_labels(dataset):
    all_features = []
    all_labels = []

    for images, labels in dataset:
        preprocessed_images = keras.applications.inception_v3.preprocess_input(images)
        features = base_model.predict(preprocessed_images)
        all_features.append(features)
        all_labels.append(labels)

    return np.concatenate(all_features), np.concatenate(all_labels)

In [9]:
train_features, train_labels = get_features_and_labels(train_dataset)
val_features, val_labels = get_features_and_labels(val_dataset)

1/1 [==============================] - 2s 2s/step


In [16]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='models/chest-opacities-detection-with-InceptionV3-pretrained-model.keras',
        save_best_only=True,
        monitor='val_loss'
    )
]

In [17]:
history = model.fit(
    train_features, train_labels,
    epochs=30,
    validation_data=(val_features, val_labels),
    callbacks=callbacks
)

Epoch 1/30


ValueError: in user code:

    File "/Users/maron/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/Users/maron/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/maron/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/Users/maron/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/Users/maron/anaconda3/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/maron/anaconda3/lib/python3.10/site-packages/keras/engine/input_spec.py", line 295, in assert_input_compatibilit

In [11]:
import numpy as np
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt

In [12]:
conv_base = keras.applications.InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(395, 488, 3))

219055592/219055592 [==============================] - 54s 0us/step


In [3]:
def get_features_and_labels(dataset):
    all_features = []
    all_labels = []

    for images, labels in dataset:
        preprocessed_images = keras.applications.inception_v3.preprocess_input(images)
        features = conv_base.predict(preprocessed_images)
        all_features.append(features)
        all_labels.append(labels)

    return np.concatenate(all_features), np.concatenate(all_labels)

In [4]:
# Path to the images
path_to_images = "training-data/dataset1/known_images"

In [5]:
train_dataset, val_dataset = keras.utils.image_dataset_from_directory(
    path_to_images,
    class_names=["normal", "sick"],
    batch_size=32,
    image_size=(395, 488),
    validation_split=0.2,
    subset="both",
    seed=200
)

Found 715 files belonging to 2 classes.
Using 572 files for training.
Using 143 files for validation.


In [6]:
train_features, train_labels = get_features_and_labels(train_dataset)
val_features, val_labels = get_features_and_labels(val_dataset)

2023-05-17 22:05:33.273924: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 1s 1s/step


In [7]:
inputs = keras.Input(shape=(395, 488, 3))
x = conv_base(inputs, training=False)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256)(x)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

In [8]:
model.compile(
    loss='binary_crossentropy',
    optimizer='rmsprop',
    metrics=[
        keras.metrics.Accuracy(),
        keras.metrics.BinaryAccuracy(),
        keras.metrics.Precision(),
        keras.metrics.AUC(),
        keras.metrics.Recall(),
        keras.metrics.TruePositives(),
        keras.metrics.FalsePositives(),
        keras.metrics.TrueNegatives(),
        keras.metrics.FalseNegatives()
    ]
)

In [9]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='models/chest-opacities-detection-with-InceptionV3-pretrained-model.keras',
        save_best_only=True,
        monitor='val_loss'
    )
]

In [10]:
history = model.fit(
    train_features, train_labels,
    epochs=30,
    validation_data=(val_features, val_labels),
    callbacks=callbacks
)

Epoch 1/30


ValueError: in user code:

    File "/Users/maron/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/Users/maron/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/maron/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/Users/maron/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/Users/maron/anaconda3/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/maron/anaconda3/lib/python3.10/site-packages/keras/engine/input_spec.py", line 295, in assert_input_compatibilit

In [ ]:
line_metrics = ['accuracy', 'binary_accuracy', 'precision', 'auc', 'recall']
confusion_matrix_metrics = ['true_positives', 'false_positives', 'true_negatives', 'false_negatives']

line_fig, line_ax = plt.subplots(figsize=(12, 8))
confusion_matrix_fig, confusion_matrix_ax = plt.subplots(figsize=(12, 8))

for metric in line_metrics:
    values = history.history[metric]
    epochs = range(1, len(values) + 1)
    line_ax.plot(epochs, values, label=metric)

line_ax.set_xlabel('Epochs')
line_ax.set_ylabel('Metric Value')
line_ax.set_title('Line Curves')
line_ax.legend()

confusion_matrix_values = [history.history[metric][-1] for metric in confusion_matrix_metrics]

confusion_matrix_ax.bar(confusion_matrix_metrics, confusion_matrix_values)
confusion_matrix_ax.set_xlabel('Metric')
confusion_matrix_ax.set_ylabel('Metric Value')
confusion_matrix_ax.set_title('Confusion Matrix Metrics')

plt.show()